In [56]:
import os
import json
from openai import OpenAI
import gradio as gr


In [57]:
MODEL_LLAMA = "llama3.2"

In [58]:
!ollama pull llama3.2
        
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [59]:
system_message = f"You are a helpful assistant for an Airline. Give short, couteous answers, no more than 1 sentence. Always be accurate. If you don't know the answer, say so."

In [60]:
def chat(message, history):
    messages = [{"role":"system","content":system_message}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(model=MODEL_LLAMA, messages=messages)
    return response.choices[0].message.content


In [61]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [62]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [63]:
tools = [{"type": "function", "function": price_function}]

In [64]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL_LLAMA, messages=messages, tools=tools)

    print(f"Message is:", messages)
    print(f"Response: {response}")
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL_LLAMA, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [65]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [66]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


In [67]:
import base64
from io import BytesIO
from PIL import Image

In [69]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-2",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [68]:
image = artist("New York City")
display(image)

NotFoundError: 404 page not found